In [2]:
import pandas as pd
import sqlite3
import sqlalchemy

In [3]:
# Error handling
try: 
    # Connect to the database
    conn = sqlite3.connect("transfermarkt.db")
    cursonr = conn.cursor()
except sqlite3.Error as e:
    # If problem
    print("Error connecting to the database", e)

In [4]:
cursor = conn.cursor()

In [4]:
print('How many players are in the database?')
cursor.execute("""
SELECT COUNT(DISTINCT player_id) [PLAYERS]
FROM PLAYERS
""")
cursor.fetchall()

How many players are in the database?


[(29455,)]

In [5]:
print('How many players with an appearance are in the database?')
cursor.execute("""
SELECT COUNT(DISTINCT player_id) [PLAYERS]
FROM APPEARANCES
""")
cursor.fetchall()

How many players have an appearance are in the database?


[(19076,)]

In [6]:
print('How many players have at least more than 1 appearance?')
cursor.execute("""
WITH TWO_OR_MORE AS (
    SELECT PLAYER_ID
        ,COUNT(DISTINCT APPEARANCE_ID) [APPS]
    FROM APPEARANCES 
    GROUP BY PLAYER_ID 
    HAVING COUNT(DISTINCT APPEARANCE_ID) > 1
)
SELECT COUNT(DISTINCT PLAYER_ID) [PLAYERS W >1 APP]
FROM TWO_OR_MORE
""")
cursor.fetchall()

How many players have at least more than 1 appearance?


[(17568,)]

In [44]:
print('What is the maximum number of appearances possible?')
cursor.execute("""
WITH APPS_PER AS (
    SELECT a.PLAYER_ID
        ,COUNT(DISTINCT a.APPEARANCE_ID) [APPS]
    FROM APPEARANCES a
    GROUP BY a.PLAYER_ID 
)
SELECT MAX(ap.APPS)
FROM APPS_PER ap
""")
cursor.fetchall()

What is the maximum number of appearances possible?


[(427,)]

In [8]:
print('Group number of players by number of appearances')
cursor.execute("""
WITH APPS_PER AS (
    SELECT a.PLAYER_ID
        ,COUNT(DISTINCT a.APPEARANCE_ID) [APPS] 
    FROM APPEARANCES a 
    GROUP BY a.PLAYER_ID 
)
SELECT CASE 
        WHEN ap.[APPS] BETWEEN 1 AND 100 THEN '1-100'
        WHEN ap.[APPS] BETWEEN 101 AND 200 THEN '101-200'
        WHEN ap.[APPS] BETWEEN 201 AND 300 THEN '201-300'
        WHEN ap.[APPS] BETWEEN 301 AND 400 THEN '301-400'
        WHEN ap.[APPS] > 400 THEN '400+'
        ELSE NULL END [APPS]
    ,COUNT(DISTINCT ap.PLAYER_ID) [PLAYERS]
FROM APPS_PER ap 
GROUP BY 1
""")
cursor.fetchall()

Group number of players by number of appearances


[('1-100', 15074),
 ('101-200', 2864),
 ('201-300', 922),
 ('301-400', 210),
 ('400+', 6)]

In [8]:
print('How far back do records go? What is the recency of the data?')
query = """
    SELECT MIN(a.DATE) [OLDEST APPEARANCE]
        ,MAX(a.DATE) [LATEST APPEARANCE]
    FROM APPEARANCES a
    UNION ALL 
    SELECT MIN(p.DATE) [OLDEST VALUAION]
        ,MAX(p.DATE) [LATEST VALUAION]
    FROM PLAYER_VALUATIONS p
    ;
"""
#cursor.fetchall()
df = pd.read_sql(query,conn)
df.head()

# appearances go back only 10 years
# valuations go back 20 years

How far back do records go? What is the recency of the data?


,OLDEST APPEARANCE,LATEST APPEARANCE
0,2014-07-01,2023-06-11
1,2003-12-09,2023-12-19


In [13]:
print('How many games have been played? What is the oldest date for a game?')
cursor.execute("""
    SELECT COUNT(DISTINCT g.GAME_ID) [TOTAL GAMES]
        ,MIN(g.DATE) [OLDEST DATE]
    FROM GAMES g
""")
cursor.fetchall()


How many games have been played? What is the oldest date for a game?


[(62379, '2012-07-03')]

In [20]:
print('How many clubs are in the database? How many clubs with their last recorded season in 2023')
cursor.execute("""
    SELECT c.LAST_SEASON
        ,COUNT(DISTINCT c.CLUB_ID) [TOTAL CLUBS]
        ,COUNT(DISTINCT CASE WHEN c.LAST_SEASON = 2023 THEN c.CLUB_ID ELSE NULL END) [2023 CLUBS]
    FROM CLUBS c
    GROUP BY 1
    ORDER BY 1
""")
cursor.fetchall()


How many clubs are in the database? How many clubs with their last recorded season in 2023


[(2012, 8, 0),
 (2013, 11, 0),
 (2014, 13, 0),
 (2015, 13, 0),
 (2016, 17, 0),
 (2017, 18, 0),
 (2018, 17, 0),
 (2019, 13, 0),
 (2020, 21, 0),
 (2021, 21, 0),
 (2022, 36, 0),
 (2023, 238, 238)]

In [40]:
print('How many leagues are in the database? How many domestic cups? How many domestic super cups? How many international cups? How many countries are represented?')
cursor.execute("""
    SELECT COUNT(DISTINCT CASE WHEN l.TYPE = 'domestic_league' AND l.COUNTRY_NAME IS NOT NULL THEN l.COMPETITION_ID ELSE NULL END) [DOMSTIC LEAGUES]
        ,COUNT(DISTINCT CASE WHEN l.TYPE = 'domestic_cup' AND l.COUNTRY_NAME IS NOT NULL THEN l.COMPETITION_ID ELSE NULL END) [DOMSTIC CUPS]
        ,COUNT(DISTINCT CASE WHEN l.SUB_TYPE = 'domestic_super_cup' AND l.COUNTRY_NAME IS NOT NULL THEN l.COMPETITION_ID ELSE NULL END) [DOMSTIC SUPER CUPS]
        ,SUM(CASE WHEN l.TYPE = 'international_cup' AND l.NAME NOT LIKE '%QUALIFYING%' THEN 1 ELSE 0 END) [DOMSTIC SUPER CUPS]
        ,COUNT(DISTINCT COUNTRY_ID) [COUNTRIES]
        --l.SUB_TYPE
        --,COUNT(DISTINCT l.COMPETITION_ID)
    FROM COMPETITIONS l
    --GROUP BY 1
""")
cursor.fetchall()

# no lower league data included in data
# no countries outside of europe included
# club world cup is included as well | competition_id = 'KLUB'

How many leagues are in the database? How many domestic cups? How many domestic super cups? How many international cups? How many world cups? How many countries are represented?


[(14, 11, 10, 5, 15)]

In [41]:
print('When is the oldest/latest game for each domestic league?')
cursor.execute("""
    SELECT c.NAME
        ,MIN(g.DATE) [EARLIEST]
        ,MAX(g.DATE) [LATEST]
    FROM COMPETITIONS c
    INNER JOIN GAMES g
    ON c.COMPETITION_ID = g.COMPETITION_ID
    WHERE c.TYPE = 'domestic_league'
    GROUP BY c.NAME
""")
cursor.fetchall()

When is the oldest/latest game for each domestic league?


[('bundesliga', '2012-08-24', '2023-05-27'),
 ('eredivisie', '2012-08-10', '2023-05-28'),
 ('jupiler-pro-league', '2012-07-28', '2023-04-23'),
 ('laliga', '2012-08-18', '2023-06-04'),
 ('liga-portugal-bwin', '2012-08-17', '2023-05-27'),
 ('ligue-1', '2012-08-10', '2023-06-03'),
 ('premier-league', '2012-08-18', '2023-05-28'),
 ('premier-liga', '2012-07-13', '2023-07-23'),
 ('scottish-premiership', '2012-08-04', '2023-04-23'),
 ('serie-a', '2012-08-25', '2023-06-04'),
 ('super-league-1', '2012-08-25', '2023-03-12'),
 ('super-lig', '2012-08-17', '2023-06-07'),
 ('superligaen', '2012-07-13', '2023-07-24')]

In [ ]:
print('Goals by year/decade, has play gotten more exciting/physical?')
cursor.execute("""
    
""")
cursor.fetchall()

In [9]:
print('When do most goals come in first tier play in England?')
cursor.execute("""
    SELECT CASE 
            WHEN ge.MINUTE BETWEEN 1 AND 10 THEN '1-10'
            WHEN ge.MINUTE BETWEEN 11 AND 20 THEN '11-20'
            WHEN ge.MINUTE BETWEEN 21 AND 30 THEN '21-30'
            --WHEN ge.MINUTE BETWEEN 31 AND 39 THEN '31-39'
            WHEN ge.MINUTE BETWEEN 31 AND 40 THEN '31-40'
            WHEN ge.MINUTE BETWEEN 41 AND 50 THEN '41-50'
            --WHEN ge.MINUTE BETWEEN 40 AND 45 THEN '40-45'
            --WHEN ge.MINUTE BETWEEN 46 AND 51 THEN '46-51'
            WHEN ge.MINUTE BETWEEN 51 AND 60 THEN '51-60'
            WHEN ge.MINUTE BETWEEN 61 AND 70 THEN '61-70'
            WHEN ge.MINUTE BETWEEN 71 AND 80 THEN '71-80'
            WHEN ge.MINUTE BETWEEN 81 AND 90 THEN '81-90'
            ELSE NULL END [SECTION]
            ,COUNT(ge.TYPE) [GOALS]
            --11483 GOALS IN ENGLAND PREM LEAGUE
            --120161 IN ALL FIRST TIER LEAGUES
            ,ROUND(1.0 * COUNT(ge.TYPE) / 
                NULLIF((SELECT COUNT(ge.TYPE) 
                        FROM COMPETITIONS c 
                        INNER JOIN GAMES g 
                        ON c.COMPETITION_ID = g.COMPETITION_ID 
                        LEFT JOIN GAME_EVENTS ge 
                        ON g.GAME_ID = ge.GAME_ID 
                            AND ge.TYPE = 'Goals' 
                            WHERE c.TYPE = 'domestic_league'
                            AND c.COUNTRY_NAME = 'England'),0),4) [%]
    FROM COMPETITIONS c
    INNER JOIN GAMES g
    ON c.COMPETITION_ID = g.COMPETITION_ID
    LEFT JOIN GAME_EVENTS ge
    ON g.GAME_ID = ge.GAME_ID
    AND ge.TYPE = 'Goals'
    WHERE c.TYPE = 'domestic_league'
    AND c.COUNTRY_NAME = 'England'
    GROUP BY 1
    ORDER BY 1 DESC
""")
cursor.fetchall()


# WHEN ge.MINUTE BETWEEN 31-40 THEN '31-40'
# WHEN ge.MINUTE BETWEEN 41-50 THEN '41-50'

When do most goals come in first tier play in England?


[('81-90', 1889, 0.1645),
 ('71-80', 1255, 0.1093),
 ('61-70', 1329, 0.1157),
 ('51-60', 1295, 0.1128),
 ('41-50', 1466, 0.1277),
 ('31-40', 1125, 0.098),
 ('21-30', 1102, 0.096),
 ('11-20', 1081, 0.0941),
 ('1-10', 941, 0.0819),
 (None, 0, 0.0)]

In [26]:
query = """
    SELECT ge.GAME_ID
        ,COUNT(DISTINCT CASE WHEN )
    FROM GAMES ge
    LEFT JOIN APPEARANCES a 
    ON ge.GAME_ID = a.GAME_ID
    LIMIT 100
;
"""

df = pd.read_sql(query,conn)
df.head()

,index,game_id,competition_id,season,round,date,home_club_id,away_club_id,home_club_goals,away_club_goals,...,home_club_manager_name,away_club_manager_name,stadium,attendance,referee,url,home_club_name,away_club_name,aggregate,competition_type
0,0,2222539,L1,2012,1. Matchday,2012-08-25,60,39,1,1,...,Christian Streich,Thomas Tuchel,Dreisamstadion,22500.0,Deniz Aytekin,https://www.transfermarkt.co.uk/sc-freiburg_1-...,SC Freiburg,1.FSV Mainz 05,1:1,domestic_league
1,1,2222541,L1,2012,1. Matchday,2012-08-25,41,4,0,1,...,Thorsten Fink,Dieter Hecking,Volksparkstadion,50123.0,Marco Fritz,https://www.transfermarkt.co.uk/hamburger-sv_1...,Hamburger SV,1.FC Nuremberg,0:1,domestic_league
2,2,2222542,L1,2012,1. Matchday,2012-08-25,24,15,2,1,...,Armin Veh,Sascha Lewandowski,Commerzbank Arena,27950.0,Thorsten Kinhöfer,https://www.transfermarkt.co.uk/eintracht-fran...,Eintracht Frankfurt,Bayer 04 Leverkusen,2:1,domestic_league
3,3,2222595,RU1,2012,3. Matchday,2012-08-05,2700,4128,1,0,...,Guus Hiddink,Rustem Khuzin,Dinamo,14000.0,Vladislav Bezborodov,https://www.transfermarkt.co.uk/anzhi-makhachk...,Anzhi Makhachkala ( -2022),Amkar Perm,1:0,domestic_league
4,4,2222636,RU1,2012,4. Matchday,2012-08-12,1083,2696,1,2,...,Miodrag Bozovic,Andrey Kobelev,Olimp-2,6800.0,Vitali Meshkov,https://www.transfermarkt.co.uk/fk-rostov_kryl...,FK Rostov,Krylya Sovetov Samara,1:2,domestic_league


In [ ]:


# What countries produce the most transfer market value? 
# Vizualize transfer value by country


# Do yellow cards impact value?
# Do yellow cards impact games?
# What about red cards?

# Is value tied to age? 
# Bucket it into percentages
